In [41]:
# Imports
import pandas
import numpy as np

In [42]:
# Helper for getting shapes of arrays
def get_shape(array):
    shape = array.shape
    rows, columns = shape
    print(f"Number of rows: {rows}, Number of columns: {columns}")
    
#For the random projection matrix R we use construction method introduced by Achlioptas
def generate_random_matrix(m, n):
    rij = np.random.choice([-1, 0, 1], size=(m, n), p=[1/6, 2/3, 1/6])
    return np.sqrt(3) * rij



In [43]:
# Getting the data if not already loaded
df_tracks = pandas.read_csv('tracks.csv',index_col=0, header=[0, 1])
df_tracks = df_tracks[df_tracks['set']['subset'] == 'medium']
df_feautures = pandas.read_csv('features.csv',index_col=0, header=[0, 1, 2])


In [45]:
# Building train, test, validation sets
# filter data only for these genres
# Hip-Hop, Pop, Folk, Rock, Experimental,
# International, Electronic, and Instrumental.
# tut nix eigentlich
df_tracks = df_tracks[df_tracks['track']['genre_top'].isin(['Hip-Hop', 'Pop', 'Folk', 'Rock', 'Experimental', 'International', 'Electronic', 'Instrumental'])]


# split track dataframe into training and test data based on column with index 30
df_tracks_train = df_tracks[df_tracks.iloc[:, 30] == 'training']
df_tracks_test = df_tracks[df_tracks.iloc[:, 30] == 'test']
df_tracks_validation = df_tracks[df_tracks.iloc[:, 30] == 'validation']

#filter features data accordingly to the track data
df_features_train = df_feautures[df_feautures.index.isin(df_tracks_train.index)]
df_features_test = df_feautures[df_feautures.index.isin(df_tracks_test.index)]
df_features_validation = df_feautures[df_feautures.index.isin(df_tracks_validation.index)]

# get X
X_train = df_features_train.values
X_test = df_features_test.values
X_validation = df_features_validation.values

# get y
y_train = df_tracks_train['track']['genre_top']
y_test = df_tracks_test['track']['genre_top']
y_validation = df_tracks_validation['track']['genre_top']

In [46]:
# evaluate shape
get_shape(X_train[0:1])

Number of rows: 1, Number of columns: 518


In [47]:
# for first test we want to have 32 bits represantations
R = generate_random_matrix(32, 518)

In [48]:
# dot product (this shows us, on which side of the hyperplane it is)
X_train_zero_dot = np.dot(X_train, R.T)

In [49]:
# now we basically say everything greateer 0 is true and therefore a one
# so we get a represantation in binary how the track data behaves with respect to R
X_train_zero_dot = X_train_zero_dot > 0
X_train_zero_dot = X_train_zero_dot.astype(int)
get_shape(X_train_zero_dot)
len(X_train_zero_dot)


Number of rows: 11912, Number of columns: 32


11912

In [50]:
# from the youtube video

buckets = {}

for i in range(len(X_train_zero_dot)):
    hash_str = ''.join(X_train_zero_dot[i].astype(str))
    if hash_str not in buckets.keys():
        buckets[hash_str] = []
    buckets[hash_str].append(i)

# print each bucket into own line for better readability
for key, value in buckets.items():
    print(key, value)

00100111001011100001000111100010 [0, 1, 12, 16, 17, 31, 32, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 56, 62, 63, 64, 65, 66, 72, 79, 80, 81, 84, 86, 87, 88, 107, 111, 114, 115, 122, 126, 141, 143, 152, 177, 178, 179, 180, 181, 203, 232, 273, 277, 301, 331, 339, 373, 377, 464, 471, 493, 500, 513, 518, 519, 521, 522, 555, 583, 584, 618, 650, 651, 652, 654, 657, 658, 665, 666, 667, 669, 671, 699, 700, 723, 724, 725, 727, 740, 756, 757, 787, 789, 790, 791, 793, 802, 806, 807, 822, 826, 827, 863, 885, 886, 887, 888, 890, 891, 892, 893, 894, 895, 896, 901, 906, 911, 920, 940, 942, 964, 966, 978, 982, 991, 993, 1003, 1004, 1005, 1006, 1007, 1010, 1013, 1016, 1017, 1020, 1023, 1031, 1035, 1036, 1046, 1047, 1053, 1057, 1059, 1060, 1067, 1078, 1079, 1081, 1089, 1093, 1095, 1096, 1097, 1105, 1107, 1111, 1121, 1124, 1142, 1155, 1174, 1176, 1177, 1179, 1210, 1211, 1214, 1246, 1251, 1260, 1261, 1262, 1263, 1265, 1266, 1280, 1288, 1289, 1290, 1293, 1294, 1295, 1296, 1298, 1305, 1320, 1

In [51]:
# testing if the results are kind of valid 

# print the corresponding y labels for each entry

# testing if the results are kind of valid 

# count the genre for each key
genre_counts = {}
for key, value in buckets.items():
    genre_counts[key] = {}
    for i in range(len(value)):
        genre = y_train.iloc[value[i]]
        if genre not in genre_counts[key]:
            genre_counts[key][genre] = 1
        else:
            genre_counts[key][genre] += 1

# sort each genre count by value
for key, value in genre_counts.items():
    genre_counts[key] = {k: v for k, v in sorted(value.items(), key=lambda item: item[1], reverse=True)}

# print the genre counts for each key
for key, value in genre_counts.items():
    print(f"Key: {key}")
    for genre, count in value.items():
        print(f"Genre: {genre}, Count: {count}")

# count in how many buckets a genre has a 90% majority
majority_percentage = 0.75
genre_majority = {}
for key, value in genre_counts.items():
    for genre, count in value.items():
        if count/len(buckets[key]) > majority_percentage:
            if genre not in genre_majority:
                genre_majority[genre] = 1
            else:
                genre_majority[genre] += 1

# print the total count of majority genres and the total amount of buckets
accuracy = sum(genre_majority.values())/len(buckets)
print(f"Accuracy: {accuracy}")






Key: 00100111001011100001000111100010
Genre: Electronic, Count: 732
Genre: Rock, Count: 464
Genre: Hip-Hop, Count: 187
Genre: Experimental, Count: 85
Genre: Folk, Count: 17
Genre: Instrumental, Count: 13
Genre: Pop, Count: 12
Key: 00100100001011110001000111110010
Genre: Rock, Count: 128
Genre: Electronic, Count: 28
Genre: Experimental, Count: 16
Genre: Instrumental, Count: 7
Genre: Hip-Hop, Count: 4
Genre: Folk, Count: 3
Genre: Pop, Count: 1
Genre: International, Count: 1
Key: 00100111001011110001000111110010
Genre: Rock, Count: 771
Genre: Electronic, Count: 260
Genre: Hip-Hop, Count: 155
Genre: Experimental, Count: 55
Genre: Folk, Count: 31
Genre: Pop, Count: 14
Genre: Instrumental, Count: 4
Key: 00110110001011110001000110100010
Genre: Rock, Count: 62
Genre: Experimental, Count: 14
Genre: Electronic, Count: 10
Genre: Folk, Count: 6
Genre: Instrumental, Count: 6
Genre: Hip-Hop, Count: 2
Genre: Pop, Count: 1
Key: 00100111001011110001000110100010
Genre: Rock, Count: 52
Genre: Electronic,

In [52]:
# assign the majority genre to each bucket
bucket_genres = {}
for key, value in genre_counts.items():
    bucket_genres[key] = list(value.keys())[0]

# use test set to evaluate the accuracy of the majority genre
X_test_zero_dot = np.dot(X_test, R.T)
X_test_zero_dot = X_test_zero_dot > 0
X_test_zero_dot = X_test_zero_dot.astype(int)


# get the majority genre for each test set entry based on the bucket_genres
majority_genres = []
for i in range(len(X_test_zero_dot)):
# find the matching bucket
    bucket_genre = bucket_genres.get(''.join(X_test_zero_dot[i].astype(str)))
    majority_genres.append(bucket_genre)


# compare the majority genre with the actual genre
correct = 0
for i in range(len(majority_genres)):
    if majority_genres[i] == y_test.iloc[i]:
        correct += 1

accuracy = correct/len(majority_genres)
print(f"Accuracy Test set: {accuracy}")

Accuracy Test set: 0.5869706840390879


In [53]:
# use combined test and validation set to evaluate the accuracy of the majority genre
X_validation_zero_dot = np.dot(X_validation, R.T)
X_validation_zero_dot = X_validation_zero_dot > 0
X_validation_zero_dot = X_validation_zero_dot.astype(int)

# get the majority genre for each test set entry based on the bucket_genres
majority_genres = []
for i in range(len(X_validation_zero_dot)):
    # find the matching bucket
    bucket_genre = bucket_genres.get(''.join(X_validation_zero_dot[i].astype(str)))
    majority_genres.append(bucket_genre)

for i in range(len(X_test_zero_dot)):
# find the matching bucket
    bucket_genre = bucket_genres.get(''.join(X_test_zero_dot[i].astype(str)))
    majority_genres.append(bucket_genre)

# compare the majority genre with the actual genre
correct = 0
for i in range(len(X_validation_zero_dot)):
    if majority_genres[i] == y_validation.iloc[i]:
        correct += 1

for i in range(len(X_test_zero_dot)):
    if majority_genres[i] == y_test.iloc[i]:
        correct += 1

accuracy = correct/len(majority_genres)
print(f"Accuracy Validation+Test set: {accuracy}")

Accuracy Validation+Test set: 0.48184818481848185
